In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
import root_numpy as rnp

import python.file_manager as fm
import ROOT

Welcome to JupyROOT 6.22/02


In [2]:
input_dir = '/eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1110pre6/DoubleElectron_FlatPt-1To100/DoubleElectron_FlatPt-1To100_PU200_v64C/'
tree_name = 'l1CaloTriggerNtuplizer_egOnly/HGCalTriggerNtuple'
maxEvents = 10
debug = 2

In [3]:
input_files = []
range_ev = (0, maxEvents)
input_files = fm.get_files_for_processing(input_dir=input_dir,
                                          tree=tree_name,
                                          nev_toprocess=maxEvents,
                                          debug=debug)

print('- will read {} files from dir {}:'.format(len(input_files), input_dir))
for file_name in input_files:
    print('        - {}'.format(file_name))

# ntuple = HGCalNtuple(input_files, tree=params.tree_name)

tree = ROOT.TChain(tree_name)

glb_files = []
for file_name in input_files:
    protocol = ''
    if '/eos/user/' in file_name:
        protocol = 'root://eosuser.cern.ch/'
    elif '/eos/cms/' in file_name:
        protocol = 'root://eoscms.cern.ch/'
    tree.Add(protocol+file_name)
    glb_files.append(protocol+file_name)

# self._tree = ROOT.TChain(tree)
#     def getEvent(self, index):
#         """Returns Event for a given index"""
#         ientry = self._tree.LoadTree(index)
#         if ientry < 0:


# for evt_idx in range(range_ev[0], range_ev[1]+1):
#     # print(evt_idx)
#     event = ntuple.getEvent(evt_idx)
#     ev_manager.read(event, debug)

    

dir already indexed, will read metadata...
[b'mkdir: illegal option -- -\n', b'usage: mkdir [-pv] [-m mode] directory ...\n', b'[eos-cp] path=/eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1110pre6/DoubleElectron_FlatPt-1To100/DoubleElectron_FlatPt-1To100_PU200_v64C/metadata.json size=71692\n', b'[eos-cp] going to copy 1 files and 71.69 kB\n', b'[eos-cp] copied 1/1 files and 71.69 kB in 0.16 seconds with 459.19 kB/s\n']
- will read 1 files from dir /eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1110pre6/DoubleElectron_FlatPt-1To100/DoubleElectron_FlatPt-1To100_PU200_v64C/:
        - /eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1110pre6/DoubleElectron_FlatPt-1To100/DoubleElectron_FlatPt-1To100_PU200_v64C/210304_174938/0000/ntuple_1.root


In [4]:
evt_idx = 0
tree.LoadTree(evt_idx)
print(tree.GetEntries())

447


In [5]:


branches = ["egammaEE_pt", "egammaEE_eta"]
entry = 2
nd_array = rnp.tree2array(tree,
                          branches=branches,
                          start=entry,
                          stop=entry+1,
                          cache_size=400000000)

# nparray = df.Range(0,1,1).AsNumpy(["egammaEE_pt", "egammaEE_eta"])


# print(nparray['egammaEE_pt'], nparray['egammaEE_eta'])



In [6]:
nd_array

array([(array([ 6.643593  ,  2.9719217 ,  3.9068716 ,  1.5156915 ,  1.300692  ,
        0.        ,  2.9264858 ,  0.2225242 ,  1.2117369 ,  0.        ,
        4.9720078 ,  3.016645  ,  2.7175887 ,  0.73760176,  0.76564455,
        0.        ,  0.5842581 ,  0.        ,  2.025458  ,  0.        ,
        1.0230687 ,  0.        ,  3.6402414 ,  1.5460535 ,  1.0310024 ,
        0.        ,  6.9713054 ,  4.120867  ,  5.0303583 ,  2.6534646 ,
        8.857585  ,  5.3984365 ,  4.317563  ,  2.2437658 ,  6.8248177 ,
        2.0740685 ,  2.9123976 ,  0.4002606 ,  2.3496022 ,  0.45174542,
        3.8635688 ,  1.7373753 ,  5.3682675 ,  2.0934558 ,  6.070648  ,
        3.5421746 ,  2.2283947 ,  0.21598236,  4.280787  ,  2.130282  ,
        4.3815894 ,  1.5497304 ,  2.0520558 ,  0.        ,  1.9534774 ,
        0.        ,  1.7097489 ,  0.        ,  2.9150255 ,  0.04426523,
       10.138936  ,  7.368551  ,  4.427791  ,  2.3982875 ,  4.7439966 ,
        2.8332458 ,  1.9536077 ,  0.        , 19.978153 

In [7]:
dir(df.Display)

NameError: name 'df' is not defined

In [ ]:
data = {}
for idx, branch in enumerate(branches):
    data[branch] = nd_array[branch][0]


In [13]:
df = ROOT.RDF.MakeNumpyDataFrame(data)

In [14]:
for colname in df.GetColumnNames():
    print (colname)

egammaEE_pt
egammaEE_eta


In [8]:
# %load ../NtupleDataFormat.py
# import math
# import collections

from __future__ import print_function
import ROOT
import pandas as pd
import root_numpy as rnp


##########
class HGCalNtuple(object):
    """Class abstracting the whole ntuple/TTree.

    Main benefit is to provide nice interface for
    - iterating over events
    - querying whether hit/seed information exists

    Note that to iteratate over the evets with zip(), you should use
    itertools.izip() instead.
    """

    def __init__(self, fileNames, tree="ana/hgc"):
        """Constructor.

        Arguments:
        fileName -- String for path to the ROOT file
        tree     -- Name of the TTree object inside the ROOT file (default: 'ana/hgc')
        """
        super(HGCalNtuple, self).__init__()
        self._tree = ROOT.TChain(tree)
        self._branches = []
        branch_blacklist = ['tc_wafer',
                            'tc_cell',
                            'tc_waferu',
                            'tc_waferv',
                            'tc_cellu',
                            'tc_cellv',
                            'gen_PUNumInt',
                            'gen_TrueNumInt']

        for file_name in fileNames:
            protocol = ''
            if '/eos/user/' in file_name:
                protocol = 'root://eosuser.cern.ch/'
            elif '/eos/cms/' in file_name:
                protocol = 'root://eoscms.cern.ch/'

            self._tree.Add(protocol+file_name)
            if len(self._branches) == 0:
                self._branches = [br for br in rnp.list_branches(protocol+file_name, tree) if br not in branch_blacklist]
        # print 'Cache size: {}'.format(self._tree.GetCacheSize())

        self._entries = self._tree.GetEntries()

    def setCache(self, learn_events=-1, entry_range=None):
        print('Resetting cache: {}'.format(self._tree.SetCacheSize(0)))
        cachesize = 400000000
        print('Setting new cache size: {}'.format(self._tree.SetCacheSize(cachesize)))
        if learn_events != -1:
            print('Setting # of entries for cache learning: {} to {}'.format(
                self._tree.SetCacheLearnEntries(learn_events), learn_events))
        else:
            print(self._tree.AddBranchToCache("*", True))
            # print self._tree.AddBranchToCache("cl_layer")
            self._tree.StopCacheLearningPhase()

        if entry_range:
            print('Setting cache entry range: {}'.format(
                self._tree.SetCacheEntryRange(entry_range[0], entry_range[-1])))
        print('Cache size: {}'.format(self._tree.GetCacheSize()))

    def PrintCacheStats(self):
        self._tree.PrintCacheStats('cachedbranches')

    def tree(self):
        return self._tree

    def nevents(self):
        return self._entries

    # def hasRawRecHits(self):
    #     """Returns true if the ntuple has raw RecHit information."""
    #     return hasattr(self._tree, "rechit_raw_pt")

    def __iter__(self):
        """Returns generator for iterating over TTree entries (events)

        Generator returns Event objects.

        """
        for jentry in range(self._entries):
            # get the next tree in the chain and verify
            ientry = self._tree.LoadTree(jentry)
            if ientry < 0:
                break
            # copy next entry into memory and verify
            nb = self._tree.GetEntry(jentry)
            if nb <= 0:
                continue

            yield Event(self._tree, jentry, self._branches)

    def getEvent(self, index):
        """Returns Event for a given index"""
        ientry = self._tree.LoadTree(index)
        if ientry < 0:
            return None
        # nb = self._tree.GetEntry(index)  # ientry or jentry?
        # if nb <= 0:
        #     None

        return Event(self._tree, index, self._branches)  # ientry of jentry?


##########
class Event(object):
    """Class abstracting a single event.

    Main benefit is to provide nice interface to get various objects
    or collections of objects.
    """

    def __init__(self, tree, entry, branches):
        """Constructor.

        Arguments:
        tree  -- TTree object
        entry -- Entry number in the tree
        """
        super(Event, self).__init__()
        self._tree = tree
        self._entry = entry
        self._branches = branches

    def entry(self):
        return self._entry

    def event(self):
        """Returns event number."""
        return self._tree.event

    def lumi(self):
        """Returns lumisection number."""
        return self._tree.lumi

    def run(self):
        """Returns run number."""
        return self._tree.run

    def eventId(self):
        """Returns (run, lumi, event) tuple."""
        return (self._tree.run, self._tree.lumi, self._tree.event)

    def eventIdStr(self):
        """Returns 'run:lumi:event' string."""
        return "%d:%d:%d" % self.eventId()

    def getDataFrame(self, prefix, fallback=None):

        branches = [br for br in self._branches
                    if br.startswith(prefix+'_') and
                    not br == '{}_n'.format(prefix)]
        names = ['_'.join(br.split('_')[1:]) for br in branches]

        if len(branches) == 0:
            if fallback is not None:
                return self.getDataFrame(prefix=fallback)
            return pd.DataFrame()

        nd_array = rnp.tree2array(self._tree,
                                  branches=branches,
                                  start=self._entry,
                                  stop=self._entry+1,
                                  cache_size=400000000)

        df = pd.DataFrame(columns=names)
        for idx, branch in enumerate(branches):
            df[names[idx]] = nd_array[branch][0]
        return df


    def getPUInfo(self):
        branches = ['gen_PUNumInt', 'gen_TrueNumInt']
        names = ['PU', 'PUTrue']
        nd_array = rnp.tree2array(self._tree, branches=branches,
                                  start=self._entry, stop=self._entry+1)
        df = pd.DataFrame(columns=names)
        for idx, name in enumerate(names):
            df[name] = [nd_array[0][idx]]
        return df


In [9]:
ntuple = HGCalNtuple(input_files, tree_name)


In [10]:
evt_idx = 0

In [11]:

event = ntuple.getEvent(evt_idx)

In [12]:
import python.collections as coll

In [13]:
coll.tkem_EE.fill(event, debug)

In [21]:
coll.tkem_EE.df.dtypes

pt           float32
energy       float32
eta          float32
phi          float32
hwQual       float32
tkIso        float32
tkIsoPV      float32
pfIso        float32
pfIsoPV      float32
looseTkID       bool
photonID        bool
dtype: object

In [22]:
coll.tkem_EE.df['test'] = 1.
coll.tkem_EE.df.test

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
301    1.0
302    1.0
303    1.0
304    1.0
305    1.0
Name: test, Length: 306, dtype: float64

In [23]:
coll.tkem_EE.df.looseTkID.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [24]:
datadict = {}
for col in coll.tkem_EE.df.columns:
    if col in ['looseTkID', 'photonID']:
        continue
    datadict[col] = coll.tkem_EE.df[col].values


In [25]:
datadict

{'pt': array([1.0471219 , 0.        , 0.9309919 , 0.        , 0.8299963 ,
        0.        , 1.8280152 , 0.        , 0.6906006 , 0.        ,
        1.4519302 , 0.        , 0.7814226 , 0.        , 2.0772543 ,
        0.04368379, 3.349274  , 0.7291381 , 7.6142917 , 2.102239  ,
        1.3601102 , 0.        , 1.1078637 , 0.        , 0.949197  ,
        0.        , 2.3532417 , 0.        , 1.8976434 , 0.        ,
        5.507132  , 2.249416  , 1.7864147 , 0.        , 0.6643612 ,
        0.        , 5.1721015 , 1.7180849 , 4.8361177 , 2.3169558 ,
        2.6669288 , 0.2981977 , 5.7898808 , 3.8040826 , 4.3599057 ,
        1.3806417 , 3.8905418 , 0.73756325, 2.363905  , 0.        ,
        1.6467872 , 0.        , 3.2733142 , 1.0909094 , 1.9897656 ,
        0.        , 3.076519  , 0.857164  , 2.0942447 , 0.        ,
        3.735676  , 1.0636082 , 2.2733889 , 0.        , 2.3596497 ,
        0.24583   , 6.138474  , 1.8592441 , 5.7122846 , 2.7330227 ,
        2.0445237 , 0.        , 1.7429491 

In [26]:
rdf = ROOT.RDF.MakeNumpyDataFrame(datadict)
for col in rdf.GetColumns():
    print(col)

AttributeError: 'RDataFrame' object has no attribute 'GetColumns'

In [ ]:
import python.selections as sel

In [ ]:
sel.eg_id_pt_ee_selections

In [ ]:
sel.eg_id_pt_ee_selections[6].selection

In [ ]:
%time rdf.Filter(sel.eg_id_pt_ee_selections[0].selection).Count().GetValue()

In [ ]:
coll.tkem_EE.cached_queries = dict()

In [ ]:
%time coll.tkem_EE.query(sel.eg_id_pt_ee_selections[0])

In [11]:
%time 
htemp = rdf.Filter(sel.eg_id_pt_ee_selections[0].selection).Histo1D(ROOT.RDF.TH1DModel('pippo'+'_pt_temp', 'EG Pt (GeV); p_{T} [GeV]', 100, 0, 100), 'pt')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


NameError: name 'rdf' is not defined

In [ ]:
%time htemp.GetValue()

In [ ]:
import numba
import numpy as np

In [ ]:
import numba


@numba.jit
def f_plain(x):
    return x * (x - 1)


@numba.jit
def integrate_f_numba(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx


@numba.jit
def apply_integrate_f_numba(col_a, col_b, col_N):
    n = len(col_N)
    result = np.empty(n, dtype="float64")
    assert len(col_a) == len(col_b) == n
    for i in range(n):
        result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
    return result


def compute_numba(df):
    result = apply_integrate_f_numba(
        df["a"].to_numpy(), df["b"].to_numpy(), df["N"].to_numpy()
    )
    return pd.Series(result, index=df.index, name="result")

@numba.jit

def fill_test(histo, col_a):
    n = len(col_a)
    for i in range(n):
        histo.Fill(col_a[i])




In [ ]:
@numba.jit
def fill_test(histo, col_a):
    n = len(col_a)
    for i in range(n):
        histo.Fill(col_a[i])



In [ ]:
@cython.boundscheck(False)
@cython.wraparound(False)
def fill_h1(hist, np.ndarray[np.double_t, ndim=1] array, np.ndarray[np.double_t, ndim=1] weights=None,
            bool return_indices=False):
    # perform type checking on python side
    cdef TH1* _hist = <TH1*> PyCObject_AsVoidPtr(hist)
    cdef SIZE_t size = array.shape[0]
    cdef np.ndarray[np.int_t, ndim=1] idx = np.empty(0, dtype=np.int)
    cdef SIZE_t i
    cdef int bin_idx
    if return_indices:
        idx = np.empty(size, dtype=np.int)
    if weights is not None:
        for i from 0 <= i < size:
            bin_idx = _hist.Fill(array[i], weights[i])
            if return_indices:
                idx[i] = bin_idx
    else:
        for i from 0 <= i < size:
            bin_idx = _hist.Fill(array[i])
            if return_indices:
                idx[i] = bin_idx
    if return_indices:
        return idx

In [ ]:
coll.tkem_EE.df.pt.values
print (len(coll.tkem_EE.df.phi.values))

In [17]:
myhisto = ROOT.TH1F('name'+'_pt', 'EG Pt (GeV); p_{T} [GeV]', 100, 0, 100)

# apply_integrate_f_numba(coll.tkem_EE.df.pt.values, coll.tkem_EE.df.eta.values, coll.tkem_EE.df.phi.values)

In [28]:
%time fill_test(myhisto, coll.tkem_EE.df.pt.values)

NameError: name 'fill_test' is not defined

In [1]:
ROOT.gROOT.ProcessLine('#include "../src/fastfilling.h"');

NameError: name 'ROOT' is not defined

In [31]:
ROOT.fillTH1Fast1(coll.tkem_EE.df.phi.values)

AttributeError: Failed to get attribute fillTH1Fast1 from ROOT

In [18]:
manager = ROOT.FillerManager()

In [19]:
pts = coll.tkem_EE.df.pt.values

manager.add_1Dhisto(myhisto, pts, coll.tkem_EE.df.looseTkID.values)

   entry: 0 value: 1.04712
   entry: 1 value: 0
   entry: 2 value: 0.930992
   entry: 3 value: 0
   entry: 4 value: 0.829996
   entry: 5 value: 0
   entry: 6 value: 1.82802
   entry: 7 value: 0
   entry: 8 value: 0.690601
   entry: 9 value: 0
   entry: 10 value: 1.45193
   entry: 11 value: 0
   entry: 12 value: 0.781423
   entry: 13 value: 0
   entry: 14 value: 2.07725
   entry: 15 value: 0.0436838
   entry: 16 value: 3.34927
   entry: 17 value: 0.729138
   entry: 18 value: 7.61429
   entry: 19 value: 2.10224
   entry: 20 value: 1.36011
   entry: 21 value: 0
   entry: 22 value: 1.10786
   entry: 23 value: 0
   entry: 24 value: 0.949197
   entry: 25 value: 0
   entry: 26 value: 2.35324
   entry: 27 value: 0
   entry: 28 value: 1.89764
   entry: 29 value: 0
   entry: 30 value: 5.50713
   entry: 31 value: 2.24942
   entry: 32 value: 1.78641
   entry: 33 value: 0
   entry: 34 value: 0.664361
   entry: 35 value: 0
   entry: 36 value: 5.1721
   entry: 37 value: 1.71808
   entry: 38 value: 4.

In [20]:
manager.fill()

SegmentationViolation: void FillerManager::fill() =>
    SegmentationViolation: segfault in C++; program state was reset

# of events: 35053564126224
FILL histo: name_pt: 0x7f862ce3a1c0
   entry: 0 value: [/opt/root/root6/lib/libCore.so] TUnixSystem::DispatchSignals(ESignals) (no debug info)
[/usr/lib/system/libsystem_platform.dylib] _sigtramp (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] tiny_free_no_lock (no debug info)
[<unknown binary>] (no debug info)
[<unknown binary>] (no debug info)
[/opt/root/root6/lib/libcppyy_backend3_8.so] WrapperCall(long, unsigned long, void*, void*, void*) (no debug info)
[/opt/root/root6/lib/libcppyy3_8.so] CPyCppyy::(anonymous namespace)::VoidExecutor::Execute(long, void*, CPyCppyy::CallContext*) (no debug info)
[/opt/root/root6/lib/libcppyy3_8.so] CPyCppyy::CPPMethod::ExecuteFast(void*, long, CPyCppyy::CallContext*) (no debug info)
[/opt/root/root6/lib/libcppyy3_8.so] CPyCppyy::CPPMethod::ExecuteProtected(void*, long, CPyCppyy::CallContext*) (no debug info)
[/opt/root/root6/lib/libcppyy3_8.so] CPyCppyy::CPPMethod::Call(CPyCppyy::CPPInstance*&, _object*, _objec

 *** Break *** segmentation violation


In [ ]:
pts